# Gemini: Una descripción general de los escenarios multimodales

## Visión General

### Gemini

Gemini es una familia de modelos generativos de IA desarrollados por Google DeepMind y diseñados para casos de uso multimodales. La API Gemini da acceso a los modelos Gemini Pro y Gemini Pro Vision

### API Vertex AI Gemini

La API Vertex AI Gemini nos da una interfaz unificada para interactuar con modelos Gemini. Actualmente existen dos modelos disponibles en la API Gemini:

- **Modelos Gemini Pro** (`gemini-pro`): Diseñado para manejar tareas de lenguaje natural, chat multiturno de texto y código y generación de código.
- **Modelo Gemini Pro Vision** (`gemini-pro-vision`): Soporta prompts multimodales. Puedes incluir texto, imágenes y video en los prompt y obtener respuestas en texto o código.

Puedes interactuar con la API Gemini usando los siguientes métodos:

- Usando [Vertex AI Studio](https://cloud.google.com/generative-ai-studio) para pruebas rápidas y generación de contenidos de texto.
- Usando el SDK de Vertex AI

Esta libreta se concentra en el uso de **Vertex AI SDK para Python** para utilizar la API Vertex AI Gemini.

Para obtener más información, consulta la documentación [IA Generativa en Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).

### Objetivos

Esta libreta demuestra una variedad de casos de uso multimodal para los que se puede utilizar Gemini.

#### Casos de uso multimodales

En comparación con los LLM (*Large Language Model*, modelos grandes de lenguaje) de sólo texto, la multimodalidad de Gemini Pro Vision se puede utilizar para muchos casos de uso nuevos:

Ejemplos de casos de uso con **texto e imágenes** como entrada:

- Detectar objetos en fotografías
- Comprender las pantallas y las interfaces
- Compresión de dibujos y abstracciones
- Comprender gráficos y diagramas
- Recomendación de imágenes basada en las preferencias del usuario
- Comparar imágenes en busca de similitudes, anomalías o diferencias

Ejemplos de casos de uso con **texto y video** como entrada:

- Generar la descripción de un video
- Extraer etiquetas de objetos a lo largo de un video
- Extraer momentos destacados/mensajes de un video

### Costos

Este tutorial usa los siguientes componentes de Google Cloud que pueden generar costos en su factura:

- Vertex AI

Para mayores detalles puedes revisar los [precios de Vertex AI](https://cloud.google.com/vertex-ai/pricing) y usar la [calculadora de precios](https://cloud.google.com/products/calculator/) para generar una estimación de costos con base en el uso del proyecto.

## Primeros pasos

### Instala el SDK de Vertex AI

**Importante:** sólo ejecuta la siguiente línea si estás trabajando en Google Colab, o bien si es la primera libreta que ejecutas en un entorno de desarrollo local. Si ya instalaste las librerías en tu equipo local, no es necesario volver a hacerlo.

In [ ]:
# ! pip3 install --upgrade google-cloud-aiplatform

### **Sólo para uso en Colab - Autentica tu ambiente de trabajo**

En el caso que estés ejecutando esta libreta en un Google Colab, descomenta la siguiente celda para realizar la autenticación de la sesión en la libreta con Google Cloud. Este paso es importante **para la utilización en Colab**, así podemos garantizar que las llamadas a las APIs de Google Cloud funcionen sin problemas.

In [ ]:
"""
import sys
# Autenticacion adicional se necesita en Google Colab
if "google.colab" in sys.modules:
    # Autentica el usuario con Google Cloud
    from google.colab import auth

    auth.authenticate_user()
"""

### **Sólo para uso en Colab - define el proyecto en Google Cloud a utilizar**

En el caso que estés ejecutando esta libreta en Google Colab, descomenta la siguiente celda para definir qué proyecto en Google Cloud será usado por Colab en la ejecución de esta libreta. De lo contrario, continua con los siguientes pasos.

In [ ]:
"""
if "google.colab" in sys.modules:
    # Define la información del proyecto
    PROJECT_ID = "your-project-id" # @param {type:"string"}
    LOCATION = "us-central1" # @param {type:"string"}

    # Inicializa Vertex AI
    import vertexai

    vertexai.init(project=PROJECT_ID, location=LOCATION)
"""

### **Sólo para uso en entorno de desarrollo local - configurar las credenciales de aplicación por defecto (ADC)**

<div class="alert alert-block alert-warning">
  <strong>
    ⚠️ Importante:
  </strong> Este proceso se debe realizar sólo una vez. Si ejecutaste
  una libreta con anterioridad, no es necesario ejecutarlo de nuevo.
</div>

En el caso que estés ejecutando esta libreta en un entorno de desarrollo local, necesitarás configurar las credenciales de aplicaciones por defecto (ADC). ADC es una estrategia usada por las librerías de Google para automáticamente encontrar credenciales de acceso basadas en el entorno de la aplicación. Para ello, sigue los siguientes pasos:

- [Instala y configura la herramienta gcloud CLI](https://cloud.google.com/sdk/docs/install)
- Crea el archivo de credenciales

In [ ]:
#! gcloud auth application-default login

### Importa las bibliotecas necesarias

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
warnings.simplefilter('ignore', UserWarning)

from vertexai.generative_models import (
  GenerationConfig,
  GenerativeModel,
  Image,
  Part
)

## Importa el modelo `Gemini 1.0 Pro Vision`

Gemini Pro Vision (`gemini-1.0-pro-vision`) es un modelo multimodal que admite indicaciones multimodales. Puede incluir texto, imágenes y videos en sus solicitudes de avisos y obtener respuestas en texto o código.

In [3]:
multi_model = GenerativeModel('gemini-1.0-pro-vision')

### Define algunas funciones auxiliares

Define algunas funciones auxiliares. Estas funciones **no** son necesarias para trabajar con Gemini Pro Vision, las usamos para visualizar las imágenes y videos que mandamos como parte de la entrada aquí en la libreta de Python.

In [5]:
import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps

def display_images(
  images: typing.Iterable[Image],
  max_width: int = 600,
  max_height: int = 350
) -> None:
  for image in images:
    pil_image = typing.cast(PIL_Image.Image, image._pil_image)

    if pil_image.mode != 'RGB':
      # RGB es soportado por todos los entornos Jupyter
      pil_image = pil_image.convert('RGB')

    image_width, image_height = pil_image.size

    if max_width < image_width or max_height < image_height:
      # Redimensiona para mostrar la imagen más pequeña en la libreta
      pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))

    IPython.display.display(pil_image)


def get_image_bytes_from_url (image_url: str) -> bytes:
  with urllib.request.urlopen(image_url) as response:
    response = typing.cast(http.client.HTTPResponse, response)
    image_bytes = response.read()

  return image_bytes


def load_image_from_url (image_url: str) -> Image:
  image_bytes = get_image_bytes_from_url(image_url)
  return Image.from_bytes(image_bytes)


def display_content_as_image (content: str | Image | Part) -> bool:
  if not isinstance(content, Image):
    return False

  display_images([ content ])

  return True


def display_content_as_video (content: str | Image | Part) -> bool:
  if not isinstance(content, Part):
    return False

  part = typing.cast(Part, content)
  file_path = part.file_data.file_uri.removeprefix('gs://')
  video_url = f'https://storage.googleapis.com/{file_path}'

  IPython.display.display(IPython.display.Video(video_url, width=600))

  return True


def print_multimodal_prompt (contents: list[ str | Image | Part ]):
  for content in contents:
    if display_content_as_image(content):
      continue
    if display_content_as_video(content):
      continue
    
    print(content)


## Comprender el contexto en varias imágenes

Una de las capacidades de Gemini es poder razonar a través de múltiples imágenes.

Aquí hay un ejemplo del uso de Gemini para calcular el costo total de los alimentos usando una imagen de frutas y una lista de precios:


In [ ]:
image_grocery_url = 'https://storage.googleapis.com/jcaguilar-dot-dev/frutas.png'
image_prices_url = 'https://storage.googleapis.com/jcaguilar-dot-dev/precios.png'
image_grocery = load_image_from_url(image_grocery_url)
image_prices = load_image_from_url(image_prices_url)

instrucciones = 'Instrucciones: Considera la siguiente imagen que contiene frutas'
prompt1 = '¿Cuánto pagaré por las frutas considerando esta lista de precios?'
prompt2 = """
Responde la pregunta a través de estos pasos:
Paso 1: Identifica qué tipo de fruta hay en la primera imagen.
Paso 2: Cuenta la cantidad de cada fruta.
Paso 3: Por cada artículo en la primera imagen, verifica el precio del artículo en la tabla de precios
Paso 4: Calcula el precio subtotal de cada tipo de fruta.
Paso 5: Calcula el precio total de las frutas usando los subtotales.
Paso 6: Si pago con un billete de $100, ¿cuánto cambio recibiré?

Responde y describe los pasos seguidos.
"""

contenidos = [
  instrucciones,
  image_grocery,
  prompt1,
  image_prices,
  prompt2
]

respuesta = multi_model.generate_content(contenidos, stream=True)

print('---------- Prompt ----------')
print_multimodal_prompt(contenidos)

print('\n---------- Respuesta ----------')
for fragmento in respuesta:
  print(fragmento.text, end='')


## Comprender las pantallas y las interfaces

Gemini también puede extraer información de las pantallas de los dispositivos, interfaces de usuario, capturas de pantalla, íconos y diseños.

Por ejemplo, si ingresas una imagen de una estufa, puedes perdirle a Gemini que proporcione instrucciones para ayudar al usuario a navegar por la interfaz y responder en diferentes idiomas:

In [ ]:
image_stove_url = 'https://storage.googleapis.com/jcaguilar-dot-dev/stove.jpg'
image_stove = load_image_from_url(image_stove_url)

prompt = """
¿Cómo puedo configurar el reloj en este dispositivo?
Si las instrucciones incluyen botones, explica también dónde están ubicados
físicamente esos botones.

Por favor, proporciona tres versiones de instrucciones: en español, en portugués
y en francés.
"""

contenido = [image_stove, prompt]

respuesta = multi_model.generate_content(contenido, stream=True)

print('\n---------- Prompt ----------')
print_multimodal_prompt(contenido)

print('\n---------- Respuesta ----------')
for fragmento in respuesta:
  print(fragmento.text, end='')

**Nota:** Es posible que la respuesta no sea del todo precisa, ya que el modelo puede _alucinar_; sin embargo, el modelo es capaz de identificar la ubicación de los botones y traducirla en una única consulta. Para mitigar las alucinaciones, un enfoque es fundamentar el LLM con técnicas como RAG, que está fuera del alcance de este tutorial.

## Comprender la relación entre entidades en diagramas técnicos

Gemini tiene capacidades multimodales que le permiten comprender diagramas y realizar pasos prácticos como optimización o generación de código. Este ejemplo demuestra como Gemini puede descifrar un diagrama de entidad-relación (ER), comprender las relaciones entre tablas, identificar los requisitos de optimización en un entorno específico como BigQuery e incluso generar el código correspondiente.

In [ ]:
image_er_url = 'https://storage.googleapis.com/jcaguilar-dot-dev/db.png'
image_er = load_image_from_url(image_er_url)

prompt = "Por favor, documenta las entidades y relaciones en este diagrama de entidad-relación."

contenido = [image_er, prompt]

# Usa una configuración más determinista con baja temperatura
config = GenerationConfig(
  temperature=0.1,
  top_p=0.8,
  top_k=40,
  candidate_count=1,
  max_output_tokens=2048
)

respuesta = multi_model.generate_content(
  contenido, 
  generation_config=config, 
  stream=True
)

print('\n---------- Prompt ----------')
print_multimodal_prompt(contenido)

print('\n---------- Respuesta ----------')
for fragmento in respuesta:
  print(fragmento.text, end='')

## Recomendaciones basadas en múltiples imágenes

Gemini puede comparar imágenes y proporcionar recomendaciones. Esto puede resultar útil en sectores como el comercio electrónico y el comercio minorista.

A continuación se muestra un ejemplo de cómo elegir qué par de gafas serían las más adecuadas para una cara ovalada:

In [ ]:
image_glasses_url1 = 'https://storage.googleapis.com/jcaguilar-dot-dev/glasses1.jpg'
image_glasses_url2 = 'https://storage.googleapis.com/jcaguilar-dot-dev/glasses2.jpg'
image_glasses1 = load_image_from_url(image_glasses_url1)
image_glasses2 = load_image_from_url(image_glasses_url2)

prompt1 = """
¿Cuál de estas gafas me recomiendas según la forma de mi cara?
Tengo una cara ovalada.
---
Gafas 1:
"""
prompt2 = """
---
Gafas 2:
"""
prompt3 = """
---
Explica cómo tomas esta decisión.
Proporciona tu recomendación según la forma de mi cara y el razonamiento
para cada una.
"""

contenidos = [
  prompt1,
  image_glasses1,
  prompt2,
  image_glasses2,
  prompt3
]

respuesta = multi_model.generate_content(contenidos, stream=True)

print('\n---------- Prompt ----------')
print_multimodal_prompt(contenidos)

print('\n---------- Respuesta ----------')
for fragmento in respuesta:
  print(fragmento.text, end='')


## Similitudes y diferencias

Los modelos Gemini pueden comparar imágenes e identificar similitudes o diferencias entre objetos.

El siguiente ejemplo muestra dos escenas de Marienplatz en Münich, Alemania, que son ligeramente diferentes. Gemini puede comparar imágenes y encontrar similitudes/diferencias.

In [ ]:
image_landmark1_url = 'https://storage.googleapis.com/jcaguilar-dot-dev/landmark1.jpg'
image_landmark2_url = 'https://storage.googleapis.com/jcaguilar-dot-dev/landmark2.jpg'
image_landmark1 = load_image_from_url(image_landmark1_url)
image_landmark2 = load_image_from_url(image_landmark2_url)

prompt1 = """
Considera las siguientes imágenes:
Imagen 1:
"""
prompt2 = """
Imagen 2:
"""
prompt3 = """
Responde las siguientes preguntas en una frase corta.

1. ¿Qué se muestra en la Imagen 1?
2. ¿En qué se parecen las dos imágenes?
3. ¿Cuál es la diferencia entre la Imagen 1 y la Imagen 2 en términos de contenido?
"""

contenidos = [
  prompt1,
  image_landmark1,
  prompt2,
  image_landmark2,
  prompt3
]

config = GenerationConfig(
  temperature=0.0,
  top_p=0.2,
  top_k=40,
  candidate_count=1,
  max_output_tokens=2048
)

respuesta = multi_model.generate_content(
  contenidos,
  generation_config=config,
  stream=True
)

print('\n---------- Prompt ----------')
print_multimodal_prompt(contenidos)

print('\n---------- Respuesta ----------')
for fragmento in respuesta:
  print(fragmento.text, end='')

## Generando una descripción de video

Gemini también puede extraer etiquetas de un video:

In [ ]:
prompt = """
¿Qué se muestra en este video?
¿Dónde debería ir para ver esto?
¿Cuáles son otros 5 lugares en el mundo que se parecen a este?
"""

video = Part.from_uri(
  uri='gs://jcaguilar-dot-dev/mediterranean.mp4',
  mime_type='video/mp4'
)

contenidos = [prompt, video]

respuesta = multi_model.generate_content(contenidos, stream=True)

print('\n---------- Prompt ----------')
print_multimodal_prompt(contenidos)

print('\n---------- Respuesta ----------')
for fragmento in respuesta:
  print(fragmento.text, end='')


> Puedes confirmar que la ubicación es Antalya, Turquía, visitando la página de Wikipedia: https://en.wikipedia.org/wiki/Antalya

## Extraer etiquetas de objetos a lo largo del video

Gemini también puede extraer etiquetas de un video

In [ ]:
prompt = """
Responde las siguientes preguntas usando solo el video:
- ¿Qué hay en el video?
- ¿Cuál es la acción en el video?
- ¿Puedes proporcionar 10 etiquetas principales para este video?
"""

video = Part.from_uri(
  uri='gs://jcaguilar-dot-dev/photo.mp4',
  mime_type='video/mp4'
)

contenidos = [prompt, video]

respuesta = multi_model.generate_content(contenidos, stream=True)

print('\n---------- Prompt ----------')
print_multimodal_prompt(contenidos)

print('\n---------- Respuesta ----------')
for fragmento in respuesta:
  print(fragmento.text, end='')

## Hacer más preguntas sobre un video

A continuación se muestra otro ejemplo para usar con Gemini en este escenario de preguntas en video.

> **Nota:** Aunque el video contiene audio, Gemini actualmente no admite entrada de audio y sólo responderá según el video.

In [ ]:
prompt = """
¿Qué línea es esta?
¿A dónde va esto?
¿Cuáles son las estaciones/paradas?
"""

video = Part.from_uri(
  uri='gs://jcaguilar-dot-dev/train.mp4',
  mime_type='video/mp4'
)

contenidos = [prompt, video]

respuesta = multi_model.generate_content(contenidos, stream=True)

print('\n---------- Prompt ----------') 
print_multimodal_prompt(contenidos)

print('\n---------- Respuesta ----------')
for fragmento in respuesta:
  print(fragmento.text, end='')

> Puedes confirmar que esta es efectivamente la Línea Confederada en Wikipedia aquí: https://en.wikipedia.org/wiki/Confederation_Line